In [ ]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten,Dropout
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!unzip "/content/drive/MyDrive/Projects/DogBreedIdentifierRes/images.zip" -d "/content/images"

# Loading images from the dataset

In [ ]:
#Errors were coming before I added r before the directory path
train_DIR = r"/content/images/images/train"
test_DIR = r"/content/images/images/test"
#classes = [ "Airedale", "Australian Terrier", "Basenji", "Basset", "Beagle", "Bedlington Terrier", "Bernese Mountain Dog", "Black", "Blenheim Spaniel", "Bloodhound", "Border Collie", "Border Terrier", "Boston Bull", "Boxer", "Brabancon Griffon", "Briard", "Brittany Spaniel", "Bull Mastiff", "Cairn", "Cardigan", "Chesapeake Bay Retriever", "Chihuahua", "Chow", "Cocker Spaniel", "Collie", "Curly", "Dhole", "Dingo", "Doberman", "English Foxhound", "English Springer", "EntleBucher", "Eskimo Dog", "French Bulldog", "German Shepherd", "German Short", "Giant Schnauzer", "Golden Retriever", "Gordon Setter", "Great Pyrenees", "Groenendael", "Ibizan Hound", "Irish Water Spaniel" ]
#classes = [ "Beagle", "Black", "Blenheim Spaniel", "Bloodhound",  "Boxer", "Brittany Spaniel", "Bull Mastiff", "Chesapeake Bay Retriever", "Chihuahua", "Dhole", "Dingo", "Doberman", "English Foxhound", "French Bulldog", "German Shepherd", "Golden Retriever", "Great Dane", "Italian Greyhound",  "Kelpie", "Kuvasz", "Labrador Retriever", "Miniature Poodle", "Norwich Terrier", "Papillon", "Pug",  "Rottweiler",  "Siberian Husky", "Standard Poodle", "Tibetan Terrier", "Toy Poodle", "Toy Terrier", "Yorkshire Terrier"]
#classes = ["Affenpinscher", "Afghan Hound", "African Hunting Dog", "Airedale", "American Staffordshire Terrier", "Appenzeller", "Australian Terrier", "Basenji", "Basset", "Beagle", "Bedlington Terrier", "Bernese Mountain Dog", "Black", "Blenheim Spaniel", "Bloodhound", "Bluetick", "Border Collie", "Border Terrier", "Borzoi", "Boston Bull", "Bouvier Des Flandres", "Boxer", "Brabancon Griffon", "Briard", "Brittany Spaniel", "Bull Mastiff", "Cairn", "Cardigan", "Chesapeake Bay Retriever", "Chihuahua", "Chow", "Clumber", "Cocker Spaniel", "Collie", "Curly", "Dandie Dinmont", "Dhole", "Dingo", "Doberman", "English Foxhound", "English Setter", "English Springer", "EntleBucher", "Eskimo Dog", "Flat", "French Bulldog", "German Shepherd", "German Short", "Giant Schnauzer", "Golden Retriever", "Gordon Setter", "Great Dane", "Great Pyrenees", "Greater Swiss Mountain Dog", "Groenendael", "Ibizan Hound", "Irish Setter", "Irish Terrier", "Irish Water Spaniel", "Irish Wolfhound", "Italian Greyhound", "Japanese Spaniel", "Keeshond", "Kelpie", "Kerry Blue Terrier", "Komondor", "Kuvasz", "Labrador Retriever", "Lakeland Terrier", "Leonberg", "Lhasa", "Malamute", "Malinois", "Maltese Dog", "Mexican Hairless", "Miniature Pinscher", "Miniature Poodle", "Miniature Schnauzer", "Newfoundland", "Norfolk Terrier", "Norwegian Elkhound", "Norwich Terrier", "Old English Sheepdog", "Otterhound", "Papillon", "Pekinese", "Pembroke", "Pomeranian", "Pug", "Redbone", "Rhodesian Ridgeback", "Rottweiler", "Saint Bernard", "Saluki", "Samoyed", "Schipperke", "Scotch Terrier", "Scottish Deerhound", "Sealyham Terrier", "Shetland Sheepdog", "Shih", "Siberian Husky", "Silky Terrier", "Soft", "Staffordshire Bullterrier", "Standard Poodle", "Standard Schnauzer", "Sussex Spaniel", "Tibetan Mastiff", "Tibetan Terrier", "Toy Poodle", "Toy Terrier", "Vizsla", "Walker Hound", "Weimaraner", "Welsh Springer Spaniel", "West Highland White Terrier", "Whippet", "Wire", "Yorkshire Terrier"]
img_size = 256

3 models were trained on different classes to account for 90 breeds of dogs as depicted below

In [ ]:
#classes = [  "African Hunting Dog", "Airedale", "American Staffordshire Terrier", "Appenzeller", "Australian Terrier", "Basenji", "Beagle", "Bedlington Terrier", "Bernese Mountain Dog", "Black", "Blenheim Spaniel", "Bloodhound", "Border Collie", "Border Terrier", "Borzoi", "Boston Bull", "Boxer", "Brabancon Griffon", "Briard", "Brittany Spaniel", "Bull Mastiff", "Cardigan", "Chihuahua", "Chow", "Cocker Spaniel",  "Dandie Dinmont", "Dhole", "Dingo", "Doberman", "English Foxhound"]

In [ ]:
#classes = [ "EntleBucher", "Eskimo Dog", "Flat", "French Bulldog", "German Shepherd", "Giant Schnauzer", "Golden Retriever", "Great Dane", "Great Pyrenees", "Greater Swiss Mountain Dog", "Groenendael", "Ibizan Hound", "Irish Terrier", "Italian Greyhound", "Keeshond", "Kelpie", "Kerry Blue Terrier", "Komondor", "Kuvasz", "Labrador Retriever", "Lakeland Terrier", "Leonberg", "Malamute", "Malinois", "Maltese Dog", "Mexican Hairless", "Miniature Pinscher", "Miniature Poodle", "Miniature Schnauzer", "Norfolk Terrier"]

In [ ]:
classes = ["Norwegian Elkhound", "Norwich Terrier", "Old English Sheepdog", "Otterhound", "Papillon", "Pekinese", "Pembroke", "Pomeranian", "Pug", "Redbone", "Rhodesian Ridgeback", "Rottweiler", "Saint Bernard", "Saluki", "Samoyed", "Schipperke", "Scotch Terrier", "Scottish Deerhound", "Sealyham Terrier", "Shetland Sheepdog", "Siberian Husky", "Staffordshire Bullterrier", "Standard Poodle", "Standard Schnauzer", "Tibetan Mastiff", "Toy Poodle", "Vizsla", "Walker Hound", "West Highland White Terrier", "Yorkshire Terrier"]

In [ ]:
len(classes)

30

In [ ]:
X_train = []
y_train = []
for cls in classes:
    path = os.path.join(train_DIR,cls)
    cls_num = classes.index(cls)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array, (img_size, img_size))
        X_train.append(new_array)
        y_train.append(cls_num)

In [ ]:
X_test = []
y_test = []
for cls in classes:
    path = os.path.join(test_DIR,cls)
    cls_num = classes.index(cls)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        new_array = cv2.resize(img_array, (img_size, img_size))
        X_test.append(new_array)
        y_test.append(cls_num)

In [ ]:
print(len(X_test))
print(len(X_train))

750
4463


In [ ]:
#Function to plot images
def plot_sample(X,y,index):
	plt.figure(figsize = (30,2)) #to determine the size of image to be shown
	plt.imshow(X[index])
	plt.xlabel(classes[y[index]])

In [ ]:
#for i in range(0,(len(classes)-1)*25, 25):
#  plot_sample(X_test, y_test, i)

In [ ]:
#Converting the list into an np_array for further processing
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
X_train = X_train.astype('short')
y_train = np.array(y_train)
y_train = y_train.astype('short')

X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
X_test = X_test.astype('short')
y_test = np.array(y_test)
y_test = y_test.astype('short')

In [ ]:
#Shuffle the arrays
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [ ]:
y_train[:20]

array([ 2,  4, 15,  0,  8, 16, 12, 25, 26,  2,  6, 10, 14, 16, 11,  2,  5,
       22,  8, 17], dtype=int16)

In [ ]:
y_test[:20]

array([16, 22,  8, 14, 28, 15, 27, 29, 21, 22,  5, 18, 16, 21, 10, 26,  3,
       19,  4, 15], dtype=int16)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
X_train[0]
#Had tried adding a preprocessing layer in the Tensorflow model itself instead of here. But somehow, doing that affected accuracy negatively, hence had to keep it here

array([[[0.5137255 , 0.52156866, 0.56078434],
        [0.5294118 , 0.5372549 , 0.5764706 ],
        [0.52156866, 0.5294118 , 0.5686275 ],
        ...,
        [0.73333335, 0.73333335, 0.75686276],
        [0.7294118 , 0.7294118 , 0.75686276],
        [0.7294118 , 0.7294118 , 0.7529412 ]],

       [[0.5372549 , 0.54509807, 0.58431375],
        [0.5372549 , 0.54509807, 0.58431375],
        [0.5372549 , 0.54509807, 0.58431375],
        ...,
        [0.7254902 , 0.7254902 , 0.7529412 ],
        [0.7254902 , 0.7254902 , 0.7490196 ],
        [0.7254902 , 0.7254902 , 0.74509805]],

       [[0.52156866, 0.5294118 , 0.5686275 ],
        [0.5137255 , 0.52156866, 0.5568628 ],
        [0.5294118 , 0.5372549 , 0.5764706 ],
        ...,
        [0.7176471 , 0.7176471 , 0.7411765 ],
        [0.7137255 , 0.7137255 , 0.7372549 ],
        [0.7137255 , 0.7137255 , 0.7372549 ]],

       ...,

       [[0.74509805, 0.7529412 , 0.75686276],
        [0.7411765 , 0.7490196 , 0.7529412 ],
        [0.73333335, 0

# Model creation and training

In [ ]:
#This model gave around 89.5% accuracy for 10 categories in just 3 epochs.. For entire dataset I might want to run it on 4 epochs
#4th epoch gave accuracy 90.6% and might have overfitted model a little
#for entire dataset use no more than 4 epochs... 4 is good for epoch count... 3 will give similar results just a little less in accuracy
base_model = InceptionV3(weights='imagenet', include_top = False, input_shape = (img_size, img_size, 3))


cnn = models.Sequential()
#cnn.add(layers.Rescaling(1./255)),#Preprocessing layer for images... divide all input by 255
#Do not use the above preprocessing layers it negatively impacted accuracy greatly... Not sure why
cnn.add(base_model)
cnn.add(GlobalAveragePooling2D())
cnn.add(Dropout(0.3))
cnn.add(Dense(512,activation = 'relu'))
cnn.add(Dense(512,activation = 'relu'))
cnn.add(Dense(len(classes),activation = 'softmax'))

cnn.layers[0].trainable = False

In [ ]:
cnn.compile(
    optimizer = 'adam', #there are various optimizers - SGD, adam, enadam etc... Adam is popular and gives better accuracy
    loss='sparse_categorical_crossentropy', #if you want your result to be a value, use spare_categorical
    #categorical is used if want your output to be an array in which the index of class predicted has value 1 and all other indices of array 0
    metrics=['accuracy']
)

In [ ]:
cnn.fit(X_train, y_train, epochs=7) #epoch is number of iterations the model is trained on using the training data.. ie number of passes of input upon algorithm
#NOTE - Before reshaping the y_train (reducing it to 1D), the accuracy was very low (reached 0.01 in 10 epochs)
#Always ensure that output is in 1D else accuracy will be greatly impacted.

Epoch 1/7
140/140 [==============================] - 28s 101ms/step - loss: 0.7460 - accuracy: 0.7943
Epoch 2/7
140/140 [==============================] - 13s 91ms/step - loss: 0.3433 - accuracy: 0.8924
Epoch 3/7
140/140 [==============================] - 13s 92ms/step - loss: 0.2835 - accuracy: 0.9102
Epoch 4/7
140/140 [==============================] - 13s 93ms/step - loss: 0.2235 - accuracy: 0.9274
Epoch 5/7
140/140 [==============================] - 13s 95ms/step - loss: 0.1955 - accuracy: 0.9341
Epoch 6/7
140/140 [==============================] - 13s 96ms/step - loss: 0.1431 - accuracy: 0.9536
Epoch 7/7
140/140 [==============================] - 13s 96ms/step - loss: 0.1468 - accuracy: 0.9514


# Evaluation

In [ ]:
cnn.evaluate(X_test,y_test)

24/24 [==============================] - 5s 160ms/step - loss: 0.5482 - accuracy: 0.8707


[0.5481805801391602, 0.8706666827201843]

In [ ]:
plot_sample(X_test, y_test,3)

In [ ]:
y_pred = cnn.predict(X_test)
y_pred[:5] #The predicted class is the class with the maximum value (we have one value for each of 10 categories in predicted array)

In [ ]:
#the argmax fun return index of max element
pred_classes = [np.argmax(element) for element in y_pred]
pred_classes[:20]

In [ ]:
y_test[:20]

In [ ]:
print('Classification report : \n' + classification_report(y_test, pred_classes))

In [ ]:
m = confusion_matrix(y_test, pred_classes)
for i in range(0,(len(classes))):
  print(str(i) + " " + str(m[i][i]))

NameError: ignored

# Pickle the model

In [ ]:
pickle.dump(cnn, open("/content/drive/MyDrive/Projects/DogBreedIdentifierRes/img_clf(acc=87,cls=30(of last 40),imgsize=256).pkl", "wb"))

In [ ]:
#cnn = pickle.load(open("/content/drive/MyDrive/Projects/DogBreedIdentifierRes/img_clf(acc=82,classes=32,imgsize=256).pkl", "rb"))

# Convert into Tensorflow lite model for using in edge devices ( android app here )

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model = converter.convert()

with open("/content/drive/MyDrive/Projects/DogBreedIdentifierRes/img_clf(acc=87,cls=30(of last 40)).tflite", "wb") as f:
  f.write(tflite_model)